In [1]:
# Import all important libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#load_data
def load_data():
    data=pd.read_csv(r"C:\Users\luhar\Projects\Experiment-with-Progressive-Learning\data_folder\data.csv")
    label=pd.read_csv(r"C:\Users\luhar\Projects\Experiment-with-Progressive-Learning\data_folder\labels.csv")
    return data,label

data,label=load_data()
print("Data loaded")

Data loaded


In [3]:
#Print details of data
print("**************Details of data*************")
print(data.info())
print()
print("**************Details of labels*************")
print(label.info())
print(label.nunique())


**************Details of data*************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Columns: 20532 entries, Unnamed: 0 to gene_20530
dtypes: float64(20531), object(1)
memory usage: 125.5+ MB
None

**************Details of labels*************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  801 non-null    object
 1   Class       801 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB
None
Unnamed: 0    801
Class           5
dtype: int64


In [4]:
#check for null values
print("Null details")
print("************For Data***********")
print(data.isnull().sum())
print()
print("**************For label************")
print(label.isnull().sum())

Null details
************For Data***********
Unnamed: 0    0
gene_0        0
gene_1        0
gene_2        0
gene_3        0
             ..
gene_20526    0
gene_20527    0
gene_20528    0
gene_20529    0
gene_20530    0
Length: 20532, dtype: int64

**************For label************
Unnamed: 0    0
Class         0
dtype: int64


In [5]:
#dropping columns:
print("Columns dropped with all element 0")
def drop_columns(data):
    for col in data.columns:
        if data[col].sum()==0:
            data.drop(col,axis=1,inplace=True)
    return data
data=drop_columns(data)

Columns dropped with all element 0


In [6]:
for uni in label['Class'].unique():
    print(uni, len(label[label['Class']==uni]))

PRAD 136
LUAD 141
BRCA 300
KIRC 146
COAD 78


In [7]:
# Normalizing data
print("Normalization of data")
for col in data.columns[1:]:
    maxi=data[col].max()
    mini=data[col].min()
    data[col]=(data[col]-mini)/(maxi-mini)

Normalization of data


In [8]:
encoding=pd.get_dummies(label['Class'])
X=data.iloc[:,1:]

In [9]:
# Method 1 for dimension reduction
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(random_state=2,max_depth=5)
model.fit(X,encoding)
print("Dimension Reduction done")

Dimension Reduction done


In [10]:
import numpy as np
features=X.columns
importances=model.feature_importances_
indices = np.argsort(importances)[-80:]  # top 80 features
# plt.title('Feature Importances')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()

In [11]:
important_col=pd.DataFrame({"col_name":features,"importance":importances})

In [12]:
important_col=pd.DataFrame(np.sort(important_col,axis=0),columns=["col_name","importance"]).iloc[-80:]

In [13]:
final_data=pd.DataFrame()
for i in important_col.col_name:
    final_data = pd.concat([final_data,data[i] ], join = 'outer', axis = 1)
    


In [14]:
final_data=pd.concat([final_data,label['Class']],join = 'outer', axis = 1)

In [15]:
final_data = pd.concat([final_data,encoding ], join = 'outer', axis = 1)

In [16]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

# encoding train labels 
encoder.fit(final_data['Class'])
Y_train = pd.DataFrame(encoder.transform(final_data['Class']))
final_data=pd.concat([final_data,Y_train ], join = 'outer', axis = 1)
final_data.rename(columns={0:"label"},inplace=True)
final_data
print("Label Encoding done")
print("Finally, data preprocessing completed ")
print("Final version of data is stored in 'final_data' ")
print("final_data=",final_data.head())

Label Encoding done
Finally, data preprocessing completed 
Final version of data is stored in 'final_data' 
final_data=    gene_9927  gene_9928  gene_9929  gene_993  gene_9930  gene_9931  gene_9932  \
0   0.074240   0.000000        0.0  0.317736   0.000000   0.000000   0.103426   
1   0.000000   0.118172        0.0  0.611864   0.061145   0.178455   0.000000   
2   0.000000   0.000000        0.0  0.457868   0.000000   0.396726   0.000000   
3   0.000000   0.000000        0.0  0.549910   0.000000   0.315542   0.000000   
4   0.045279   0.094607        0.0  0.657807   0.092543   0.435918   0.000000   

   gene_9933  gene_9934  gene_9935  ...  gene_9997  gene_9998  gene_9999  \
0   0.549063   0.157050   0.066993  ...   0.592891        0.0   0.429999   
1   0.587582   0.000000   0.091812  ...   0.377247        0.0   0.477273   
2   0.177371   0.525680   0.000000  ...   0.346013        0.0   0.558179   
3   0.253207   0.275804   0.117650  ...   0.590198        0.0   0.625669   
4   0.617929 

In [20]:
final_data.to_csv(r"data_folder\Reduction_from_Random_Forest.csv")